In [76]:
#set backend - for pop-out interactive plots set to qt
#%matplotlib inline

#import modules
import sys
sys.path.append(r'/dls/science/groups/e02/Chris/Code/GitHub/Merlin-Medipix') # location of epsic_tools 
import hyperspy.api as hs
import matplotlib.pyplot as plt
import epsic_tools.api as epsic
import numpy as np
import os
import pandas as pd

#read parameter spreadsheed
sfn = r'/dls/e02/data/2019/cm22979-6/processing/Merlin/Merlin/20191001_15kV_ptycho/20190822_Ptychography_speadsheet_c.xlsx'
ss = pd.read_excel(sfn, skiprows =7)
#fill missing data
ps = ss.iloc[:,2:17].fillna(method='ffill')
ss.iloc[:,2:17] = ps

#initiate epsic object
ep = epsic.ptychography2D.Ptychography2D()

#get time stamp from data save path 
time_stamp = int(os.path.split(save_file)[0][-6:])
#print(time_stamp)
meta = ss[ss.Folder.isin([(time_stamp)])]
meta

,Folder,File,keV,CLA,alpha,CL,Spot,A2 (kV),Scan Mag,nominal defocus,...,Probe positions hdf5,Field of view (Ang),step size (Ang),counter depth,Saved data bit depth,frame time (ms),Notes,Unnamed: 18,Scanning step size: 0.330000 nm,Unnamed: 20
105,145557,NaN,15.0,100um,17.0,18.7,6C,1.07,3Mx,0,...,NaN,NaN,2.6,6.0,NaN,700,NaN,NaN,Ib = 121pA,NaN


In [9]:
#data set for reconstruction 

data_file_path = save_file
#'/dls/e02/data/2019/cm22979-6/processing/Merlin/Merlin/20191001_15kV_ptycho/MoS2_700C/20191002 145013/cropscan_binned_MoS2_15kV_50umClAp_3Mx_focus_A2_1p07_scan64x64_diff256x256.hdf5'

#reconstructions will be saved in the experiment ID folder in the base directory with prefix
experiment_ID = 'scancrop_64'
save_prefix = 'v0_bindiff4_1x1_modes'


#parameters for reconstruction

beam_energy = 15e3 # in volts
convergence_semi_angle = 0.0125 # in radians
camera_length = 0.12895 # in m
scan_rotation = -80.18 # in degrees
scan_step_size = 2.6e-10 # in m 
number_of_iterations = 10
defocus  = -100e-9 # in m
detector_binning = 1
source_modes = [1,1]
decay = [0.4, 0, 1]

#if running on cluster need to set interaction to 'None'
interaction = 'None' #or 'dawn' if running on dawn

In [85]:
#load into hyperspy object
d = hs.load(data_file_path, lazy = True)

#get data parameters
data_shape = d.data.shape
#scan array shape
d = None
scan_shape = data_shape[:2]
detector_shape = data_shape[2:]
print('scan shape : ', scan_shape)
print('detector shape : ',detector_shape)
binning = np.floor(515 / detector_shape[0])
print("binning : ", binning)
if binning == 1:
    mask_fp = r'//dls/e02/data/2019/cm22979-6/processing/Merlin/Reconstruction_masks/Merlin.h5'
else:
    mask_fp = r'//dls/e02/data/2019/cm22979-6/processing/Merlin/Reconstruction_masks/Merlin_bin' + str(int(binning)) +'.h5'
px_size = 55e-6 #m
#adjust for binning
adj_px_size = px_size * binning
optic_axis =  detector_shape[0] / 2

#populate ptychography object
ep.set_data_path(data_file_path)
ep.set_dead_px_path(mask_fp)
ep.set_dead_px_flag(1)
ep.set_energy(beam_energy)
ep.set_scan_size(scan_shape)
ep.set_scan_step([scan_step_size, scan_step_size])
ep.set_px_pitch([adj_px_size, adj_px_size])
ep.set_scan_rotation(scan_rotation)
ep.set_detector_pos([0 , 0 , camera_length])
ep.set_convergence( 2 * convergence_semi_angle)
ep.set_defocus(defocus)
ep.set_detector_crop(detector_shape)
ep.set_detector_bin([detector_binning, detector_binning])
ep.set_iterations(number_of_iterations)
ep.set_decay(decay)
ep.set_source_modes(source_modes)
ep.set_interaction(interaction)
#save path
data_path , _ = os.path.split(data_file_path)
save_path , dat_dir = os.path.split(data_path)

ep.set_save_prefix(save_prefix)
ep.set_experiment_ID(experiment_ID)
ep.set_base_dir(save_path)

scan shape :  (64, 64)
detector shape :  (256, 256)
binning :  2.0


In [86]:
#save json file
time_stamp = dat_dir.split(' ')[1]
json_path = save_path + '/' + experiment_ID
if not os.path.exists(json_path):
    os.makedirs(json_path)
json_file = json_path + '/' + time_stamp + '_' + experiment_ID + '_' + save_prefix + '.json'
ep.save_json(json_file)
#print(json_file)

json file saved to :  /dls/e02/data/2019/cm22979-6/processing/Merlin/Merlin/20191001_15kV_ptycho/MoS2_700C/scancrop_64/145557_scancrop_64_v0_bindiff4_1x1_modes.json


In [87]:
#run on cluster

f = []
# create folder to save cluster logs
log_path = json_path + '/logs'
if not os.path.exists(log_path):
    os.makedirs(log_path)

#make a list of all json files in folder
for (dir_path ,dir_names, file_names) in os.walk(json_path):
    print(file_names)
    f.extend(file_names)
    break
    
#send each to the cluster
n = 0
for this_file in f:
    os.system('echo /dls_sw/i13-1/scripts/Pycho/pycho_recon_single_cluster.sh ' + json_path + ' ' + this_file[:-5] + ' ' + str(n))
    os.system('cd ' + json_path +'/logs' +  '\n module load global/cluster \n qsub /dls_sw/i13-1/scripts/Pycho/pycho_recon_single_cluster.sh ' + json_path + ' ' + this_file[:-5] + ' ' + str(n))
    n = n+1
print('Recon results will be saved to : ' )
print(save_path + '/' + experiment_ID)

['145557_scancrop_64_v0_bindiff4_1x1_modes.json']
Recon results will be saved to : 
/dls/e02/data/2019/cm22979-6/processing/Merlin/Merlin/20191001_15kV_ptycho/MoS2_700C/scancrop_64


In [11]:
#save_path= '/dls/e02/data/2019/cm22979-6/processing/Merlin/Merlin/20191001_15kV_ptycho/MoS2_700C/'